# Tuning Fufi hyperparameters with W&B Sweep 🧹 🐶
The idea is automatically tuning hyperparameters with the sweep by the means of a `grid `search.
In order not to spend an eternity doing this thing, we start with a` random` search, and then put a `grid `search on it.

## Define the sweep 📑
I assume to start with a random search, that is why we take random distribution:
*   `batch_size`  ➡  Quantized log uniform. Returns `round(X / q) * q` where `X` is` log_uniform_values`. Basically, a pretty good approximation of a log_q uniform distribution
*   `ent_coef ` ➡ Discrete uniform distribution on integers. Between 0.01, that is the default value and 1, that is the thing that works best for now.
*  ` num_envs`  ➡ see batch
*   `learning_rate` ➡ see batch




In [ ]:
import wandb
import numpy as np
import random

# Define sweep config
sweep_configuration = {
    "method": "random",
    "name": "sweep_Fufi",
    "metric": {"goal": "maximize", "name": "reward"},
    "parameters": {
       "batch_size": {'distribution': 'q_log_uniform_values','q': 2,'min': 32,'max': 4096},
       "ent_coef": {'distribution': 'uniform', 'min': 0.01,'max': 1},
       "num_envs":  {'distribution': 'q_log_uniform_values','q': 2,'min': 4,'max': 16},
       "learning_rate":  {'distribution': 'q_log_uniform_values','q': 1,'min': 1.5e-7,'max': 1.5e-4}
    },
}


In [ ]:
#print the result
import pprint
pprint.pprint(sweep_configuration)

## Import main function from git repo 📡


In [ ]:
#git repo things

In [ ]:
from main_function_sweep import main

## Run main with the sweep 🏃 🧹
The `wandb.sweep` function initializes the sweep using the configuration. The `wandb.agent `function runs the sweep, executing the `sweep_main` function for each set of parameters.

📚 **Handling Parameters in Script**: In `sweep_main`, `wandb.init()` initializes a run. The script updates the args with the parameters from the sweep `(wandb.config)`, which are then passed to the main function.

📚 **Note**: Added `if __name__ == "__main__":` This ensures that main is called only when the script is executed directly, not when imported as a module.

In [ ]:
if __name__ == "__main__":
    sweep_id = wandb.sweep(sweep=sweep_configuration, project="Fufi_sweep")  # Set up the sweep

    def sweep_main():
        with wandb.init() as run:
            args = parse_args()
            # Update args with sweep parameters
            args.learning_rate = wandb.config.learning_rate
            args.batch_size = wandb.config.batch_size
            args.ent_coef = wandb.config.ent_coef
            args.num_envs = wandb.config.num_envs

            main()

In [ ]:
 wandb.agent(sweep_id, function=sweep_main, count = 30)